In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.chdir("../")
%pwd


'c:\\Users\\Bhavin\\Documents\\Project_1_Cancer_Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:Path
    training_model_path : Path
    updated_base_model_path : Path
    training_data: Path
    params_image_size: list
    params_epochs: int
    params_batch_size: int
    params_is_augmentation:bool

    

In [5]:
from src.cnnclassifier.constants import *
from src.cnnclassifier.utils.common import read_yaml,create_directories
import tensorflow

In [6]:
from src.cnnclassifier.constants import *
from src.cnnclassifier.utils.common import read_yaml,create_directories
from src.cnnclassifier.entities.config_entity import *
import tensorflow

class ConfigurationManager:
    def __init__(
        self,
        confif_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH) -> None:
        
        self.config = read_yaml(confif_filepath)
        self.params= read_yaml(params_filepath)
    
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=params.IAMGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
            params_classes=params.CLASSES,
            params_include_top=params.INCLUDE_TOP,
            params_weight=params.WEIGHTS,
            # model_summary_image=Path(config.updated_base_model_path),
            )
        
        return prepare_base_model_config
    
    def get_training_config(self) -> ModelTrainerConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params= self.params
        
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,'Data')
        
        create_directories([Path(training.root_dir)])
        
        training_config = ModelTrainerConfig(
            root_dir=Path(training.root_dir),
            training_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IAMGE_SIZE,
            params_is_augmentation=params.AUGMENTATION
            
        )
        
        return training_config
    
    

    

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten,Dense,Dropout

In [21]:
import os
import urllib.request as request
import shutil
import tensorflow as tf
import time

class Training:
    def __init__(self,config: ModelTrainerConfig) -> None:
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            # validation_split = 0.2
        )
        
        dataflow_kwargs=  dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=os.path.join(self.config.training_data,'valid'),
            shuffle = False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator= train_datagenerator.flow_from_directory(
            directory=os.path.join(self.config.training_data,"train"),
            shuffle=True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path: Path,model : tf.keras.Model):
        model.save(path)
        
        
    def train(self):
        self.get_base_model()
        self.train_valid_generator()
        # self.steps_per_epochs = self.train_generator.samples // self.train_generator.batch_size
        # self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='artifacts/training/model.weights.h5',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
        
        early_stopping_callback = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            min_delta=0,
            patience=0,
            verbose=0,
            mode="auto",
            baseline=None,
            restore_best_weights=False,
            start_from_epoch=0,
        )
        
        callbacks=[early_stopping_callback,model_checkpoint_callback]
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            # steps_per_epoch = self.steps_per_epochs,
            # validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callbacks,
            verbose=2
        )    
        
        self.save_model(
            path=self.config.training_model_path,
            model=self.model
            )
    

In [22]:
from src.cnnclassifier import CustomException,logger
import sys
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.train()
    
except Exception as e:
    raise CustomException(e,sys)

[2025-06-12 20:56:29,812] [INFO] [common:77] - yaml file:config\config.yaml loaded successfully
[2025-06-12 20:56:29,816] [INFO] [common:77] - yaml file:params.yaml loaded successfully
[2025-06-12 20:56:29,817] [INFO] [common:95] - artifacts directory is created
[2025-06-12 20:56:29,819] [INFO] [common:95] - artifacts\training directory is created
Found 72 images belonging to 4 classes.
Found 613 images belonging to 4 classes.
[2025-06-12 20:57:20,004] [WARNING] [saving_api:83] - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
39/39 - 50s - 1s/step - accuracy: 0.2496 - loss: 986.3856 - val_accuracy: 0.1806 - val_loss: 1.3897


In [10]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


TensorFlow version: 2.19.0
Num GPUs Available: 0
